# Hyperparameter optimization using HyperOpt

This tutorial describes how to perform hyperparameter optimization using `HyperOpt` tool.

To install this do `pip install hyperopt` or `conda install -c conda-forge hyperopt`


Documentation of `HyperOpt` can be found here http://hyperopt.github.io/hyperopt/


### Rice (Cammeo and Osmancik) Data Set

A total of 3810 rice grain's images were taken for the two species, processed and feature inferences were made. 7 morphological features were obtained for each grain of rice.



https://archive.ics.uci.edu/ml/datasets/Rice+%28Cammeo+and+Osmancik%29


#### Data Set Information:

Among the certified rice grown in TURKEY, the Osmancik species, which has a large planting area since 1997 and the Cammeo species grown since 2014 have been selected for the study. When looking at the general characteristics of Osmancik species, they have a wide, long, glassy and dull appearance. When looking at the general characteristics of the Cammeo species, they have wide and long, glassy and dull in appearance. A total of 3810 rice grain's images were taken for the two species, processed and feature inferences were made. 7 morphological features were obtained for each grain of rice.


#### Attribute Information:

1.) Area: Returns the number of pixels within the boundaries of the rice grain.

2.) Perimeter: Calculates the circumference by calculating the distance between pixels around the boundaries of the rice grain.

3.) Major Axis Length: The longest line that can be drawn on the rice grain, i.e. the main axis distance, gives.

4.) Minor Axis Length: The shortest line that can be drawn on the rice grain, i.e. the small axis distance, gives.

5.) Eccentricity: It measures how round the ellipse, which has the same moments as the rice grain, is.

6.) Convex Area: Returns the pixel count of the smallest convex shell of the region formed by the rice grain.

7.) Extent: Returns the ratio of the regionformed by the rice grain to the bounding box pixels.

8.) Class: Cammeo and Osmancik rices

In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('Rice_Cammeo_Osmancik.csv')
df

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent,Class
0,15231,525.578979,229.749878,85.093788,0.928882,15617,0.572896,0
1,14656,494.311005,206.020065,91.730972,0.895405,15072,0.615436,0
2,14634,501.122009,214.106781,87.768288,0.912118,14954,0.693259,0
3,13176,458.342987,193.337387,87.448395,0.891861,13368,0.640669,0
4,14688,507.166992,211.743378,89.312454,0.906691,15262,0.646024,0
...,...,...,...,...,...,...,...,...
3255,10627,411.441010,172.261276,79.841049,0.886103,10800,0.802401,1
3256,11540,427.319000,174.082809,85.417870,0.871343,11829,0.745189,1
3257,11494,433.058014,171.582459,86.878693,0.862335,11858,0.759382,1
3258,12253,437.942993,177.751053,89.389755,0.864349,12595,0.614925,1


### Split Train and test sets and feature scaling

In [4]:
df_features = df.loc[:,'Area':'Extent']
df_class = df.loc[:,'Class']

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(df_features,df_class,test_size=0.2)
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)
print(X_train_std.shape)
print(X_test_std.shape)
print("No. of samples in each class in training set ")
y_train.value_counts()

(2608, 7)
(652, 7)
No. of samples in each class in training set 


1    1313
0    1295
Name: Class, dtype: int64

### Define Hyperparameter space

In [14]:
from hyperopt import fmin, tpe, hp,  STATUS_OK, Trials
from hyperopt.pyll.base import scope
from sklearn.metrics import roc_auc_score
from functools import partial

Define the domain of hyperparameters using a python dictionary.

During each iteration of the hyperopt:

* `hp.choice(label,options)` chooses one of the provided options.

* `hp.quniform(label, low, high, q)` Returns a value like `round(uniform(low, high) / q) * q` Suitable for a discrete value with respect to which the objective is still somewhat "smooth", but which should be bounded both above and below.



* `hp.uniform(label, low, high)` Returns a value uniformly between `low` & `high`. Returns non integer values. So you will need to round it up if the hyperparameter can be integers only.



* `hp.qloguniform(label, low, high, q)`
Returns a value like `round(exp(uniform(low, high)) / q) * q`
Suitable for a discrete variable with respect to which the objective is "smooth" and gets smoother with the size of the value, but which should be bounded both above and below

Lets define two hyperparameter spaces.

In [9]:
'''
Only discrete choices are provided
'''
nn_space_1 = {
            'num_nodes': hp.choice('num_nodes',[10,25,50,75,100,125,150,175,200,250,300,350,400]),
            'optimizer': hp.choice('optimizer',[ 'sgd', 'adam']),
            'batch': hp.choice('batch',[10,20,30,50]),
            'learning_rate_init': hp.choice('learning_rate_init',[0.01,0.05,0.001,0.005,0.0005,0.0001,0.00001,0.00005]),
            'alpha': hp.choice('alpha',[0,0.01,0.05,0.001,0.005,0.0005,0.0001,0.00001,0.00005])
        }

'''
Instead of options, range is provided
'''
nn_space_2 = {
            'num_nodes': scope.int(hp.quniform('num_nodes',10,400,1)),
            'optimizer': hp.choice('optimizer',['sgd', 'adam']),
            'batch': scope.int(hp.quniform('batch',10,50,1)),
            'learning_rate_init': hp.uniform('learning_rate_init',0.00005,0.01),
            'alpha': hp.qloguniform('alpha',0,1,1)
        }

### Define objective function

* You can also put additional parameters in the function `nn1_model` below. `nn1_model` has only one hidden layer.
* `params`: the dictionary that represents the hyperparameter space.
* The value of `'loss'` returned by this function is what we want to optimize over the hyperparameter space. You can also return additional items.
* Note here if you want to maximize `accuracy` or `area under the curve`, you will need to return the negative of it. Because by default the objective function is considered to be minimized by hyperopt. 
* Minimizing `negative of accuracy` is same as maximixing `accuracy`.

In [15]:
from sklearn.neural_network import MLPClassifier

def nn1_model_obj(params,Training_features,Training_class):
    
    model = MLPClassifier(hidden_layer_sizes=(params['num_nodes'],),
                          activation='relu', # activation in the hidden layers
                          solver = params['optimizer'], # optimizer name
                          alpha = params['alpha'], # L2 regularization parameter
                          batch_size = params['batch'], # batch size
                          learning_rate = 'constant', 
                          learning_rate_init = params['learning_rate_init'], # learning rate
                          early_stopping=True, # how to stop training the model
                          validation_fraction=0.1, # how much of training set is used as validation set.
                          n_iter_no_change = 10)
    '''
    suggest to make verbose = 0 so that it doesn't print the progress of model training for every iteration of hyper parameter optimization
    '''
    model.fit(Training_features,Training_class)
    print(model)
    '''
    note: solver sgd and adam can return binary loss entropy values. but ‘lbfgs’ cannot.
    '''
    val_loss = min(model.loss_curve_)
    return {'loss': val_loss,'status': STATUS_OK, 'Trained_Model': model} 
#returning a dictionary that contains loss, status of the optimization process, and the trained model

## Run the hyperparameter optimization

* `nn1_model_obj` : objective function you want to minimize
* `X_train_std`, `y_train` : training data and class labels
* `nn1_space`: Hyperparameter space
* `algo`: bayesian algorithm to be used
* `max_evals`: number of iterations
* `trials` : object where the results are stored

The number of `max_evals` is set to 50 for illustration purpose. Higher this value, better it is. Recommended value is >= 50.

#### Run the hyperparameter optimization on first space

In [16]:
trials1 =Trials()
best_model = fmin(partial(nn1_model_obj,Training_features=X_train_std,Training_class=y_train),nn_space_1,algo=tpe.suggest,max_evals=50,trials=trials1)
print(best_model)

MLPClassifier(alpha=0.005, batch_size=30, early_stopping=True,                                                         
              hidden_layer_sizes=(25,), learning_rate_init=0.0001,
              solver='sgd')
MLPClassifier(alpha=1e-05, batch_size=20, early_stopping=True,                                                         
              hidden_layer_sizes=(350,), learning_rate_init=0.0005,
              solver='sgd')
MLPClassifier(batch_size=50, early_stopping=True, hidden_layer_sizes=(10,),                                            
              learning_rate_init=0.01)
MLPClassifier(alpha=0.05, batch_size=30, early_stopping=True,                                                          
              hidden_layer_sizes=(250,), learning_rate_init=1e-05,
              solver='sgd')
MLPClassifier(alpha=0.005, batch_size=10, early_stopping=True,                                                         
              hidden_layer_sizes=(350,), learning_rate_init=0.005)
MLPClass

### Explore the results 

* `trials1` : object where the results of hyperparameter optimization process is stored. More here http://hyperopt.github.io/hyperopt/getting-started/minimizing_functions/


In [19]:
for trial in trials1:
    print(trial)

{'state': 2, 'tid': 0, 'spec': None, 'result': {'loss': 0.2580781688266739, 'status': 'ok', 'Trained_Model': MLPClassifier(alpha=0.005, batch_size=30, early_stopping=True,
              hidden_layer_sizes=(25,), learning_rate_init=0.0001,
              solver='sgd')}, 'misc': {'tid': 0, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'alpha': [0], 'batch': [0], 'learning_rate_init': [0], 'num_nodes': [0], 'optimizer': [0]}, 'vals': {'alpha': [4], 'batch': [2], 'learning_rate_init': [5], 'num_nodes': [1], 'optimizer': [0]}}, 'exp_key': None, 'owner': None, 'version': 0, 'book_time': datetime.datetime(2022, 10, 10, 16, 36, 30, 423000), 'refresh_time': datetime.datetime(2022, 10, 10, 16, 36, 30, 741000)}
{'state': 2, 'tid': 1, 'spec': None, 'result': {'loss': 0.19343532683426828, 'status': 'ok', 'Trained_Model': MLPClassifier(alpha=1e-05, batch_size=20, early_stopping=True,
              hidden_layer_sizes=(350,), learning_rate_init=0.0005,
              solver=

each `trial` in `trials1` is a dictionary. There is a key `result` that contains another dictionary with `loss` (best loss from that specific trial), `status`, and `Trained_Model` for that specific trial.

In [22]:
'''
extracts all valid iterations of the hyperoptimization process 
'''
valid_trial_list1 = [trial for trial in trials1
                            if STATUS_OK == trial['result']['status']]
    
'''
extracts obj. function value in all valid iterations of the hyperoptimization 
'''
losses1 = [ float(trial['result']['loss']) for trial in valid_trial_list1]
print("losses are \n===========================================================\n")
print(losses1)
'''
find the one with lowest obj. function
'''
index_having_minumum_loss1 = np.argmin(losses1)

'''
extract the model with the least value of the objective function
'''
best_trial_obj1 = valid_trial_list1[index_having_minumum_loss1]
best_model1 = best_trial_obj1['result']['Trained_Model']   

losses are 

[0.2580781688266739, 0.19343532683426828, 0.17938236565904397, 0.4913525780146946, 0.18489081636310953, 0.4078929100554588, 0.3395549436595578, 0.29459584230659486, 0.19870615620153867, 0.1760823880637695, 0.3997304362460272, 0.18173281450286302, 0.7015891249622973, 0.18615612171920226, 0.2102233190520137, 0.20352574379375765, 0.22154453109416297, 0.18678078628837574, 0.18066562983869106, 0.22663097312695352, 0.1750916661049638, 0.18268984378811487, 0.17737860887387205, 0.18719113479995264, 0.18628811545991494, 0.17949029424411245, 0.18679697749962074, 0.17785041022868828, 0.18055575346704206, 0.18726694162571922, 0.17987738558039143, 0.18074141813990727, 0.21645104764055784, 0.1757084469749825, 0.17797844965554807, 0.18103241324941527, 0.174813922687737, 0.17735850285084695, 0.181196927552729, 0.3291578076566505, 0.17552519834231903, 0.2803338148619027, 0.20766562599679278, 0.1778216167777114, 0.18142511256652222, 0.5440423154506046, 0.18546298099758102, 0.253279931326728

In [23]:
print("Best hyper parameter values are")
print(best_model1)

Best hyper parameter values are
MLPClassifier(alpha=0.0005, batch_size=20, early_stopping=True,
              hidden_layer_sizes=(50,))


Perform prediction on the test set with the best model

In [24]:
from sklearn.metrics import accuracy_score
pred_classes = best_model1.predict(X_test_std) # predicted class labels
print('testing accuracy',accuracy_score(y_test,pred_classes))    

testing accuracy 0.9386503067484663


### Saving the model to a file

In [28]:
import pickle

#extracts the model corresponding to the lowest obj. function
filename = 'test_model.pkl'
pickle.dump(best_model1, open(filename, 'wb'))

### Loading the model from a file

In [29]:
# load the model from disk
loaded_model = pickle.load(open('test_model.sav', 'rb'))
print(loaded_model)

MLPClassifier(alpha=0.0005, batch_size=20, early_stopping=True,
              hidden_layer_sizes=(50,))


## Run the hyperparameter optimization on second space

In [30]:
trials2=Trials()
fmin(partial(nn1_model_obj,Training_features=X_train_std,Training_class=y_train),nn_space_2,algo=tpe.suggest,max_evals=50,trials=trials2)

# extracts all valid iterations of the hyperoptimization 
valid_trial_list2 = [trial for trial in trials2
                            if STATUS_OK == trial['result']['status']]
    
# extracts obj. function value in all valid iterations of the hyperoptimization 
losses2 = [ float(trial['result']['loss']) for trial in valid_trial_list2]
print(losses2)
# find the one with lowest obj. function
index_having_minumum_loss2 = np.argmin(losses2)
best_trial_obj2 = valid_trial_list2[index_having_minumum_loss2]
best_model2 = best_trial_obj2['result']['Trained_Model']   

pred_classes = best_model2.predict(X_test_std) # predicted class labels
print('testing accuracy',accuracy_score(y_test,pred_classes)) 

print("Best hyper parameter values are")
print(best_model2)

MLPClassifier(alpha=2.0, batch_size=17, early_stopping=True,                                                           
              hidden_layer_sizes=(57,),
              learning_rate_init=0.004288550784527108)
MLPClassifier(alpha=1.0, batch_size=23, early_stopping=True,                                                           
              hidden_layer_sizes=(106,),
              learning_rate_init=0.0032188087742628683)
MLPClassifier(alpha=2.0, batch_size=24, early_stopping=True,                                                           
              hidden_layer_sizes=(30,), learning_rate_init=0.003697422781840454,
              solver='sgd')
MLPClassifier(alpha=2.0, batch_size=19, early_stopping=True,                                                           
              hidden_layer_sizes=(201,),
              learning_rate_init=0.008594879353500812)
MLPClassifier(alpha=1.0, batch_size=12, early_stopping=True,                                                           
   

## Adjusting the parameters for the algorithm

You can use `n_EI_candidates` to control the number of candidates that TPE algorithms checks at each iteration.

In [97]:
from hyperopt import tpe
TPE = partial(
    tpe.suggest,
    # Sample 1000 candidate and select candidate that
    # has highest Expected Improvement (EI)
    n_EI_candidates=100,
    # Use 20% of best observations to estimate next
    # set of parameters
    gamma=0.2,
)


## Exercise, perform hyperparameter optimization on Random forest using HyperOpt.

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

### Random Forest hyperparameter space

In [34]:
'''
Instead of options, range is provided
'''
total_features  = 7
rf_space = {
            'n_estimators': scope.int(hp.quniform('num_nodes',50,5000,1)),
            'criterion': hp.choice('criterion',['gini', 'entropy','log_loss']),
            'max_features': scope.int(hp.quniform('max_features',1,total_features,1)),
            'min_samples_leaf' : hp.choice('min_samples_leaf',[1, 2])
        }

### Random Forest Objective Function

In [53]:
def rf_model_obj(params,Training_features,Training_class):
    
    model = RandomForestClassifier(n_estimators=params['n_estimators'],
                          criterion=params['criterion'], 
                          max_features = params['max_features'], 
                          min_samples_leaf = params['min_samples_leaf'], n_jobs=4
                        )

    '''
    calculates the score on the validation splits.
    '''
    scores = cross_val_score(model, Training_features,Training_class, cv=3, scoring = 'accuracy')
    '''
    scores= Array of scores of the estimator for each run of the cross validation.
    '''
    val_acc = -scores.mean()
    return {'loss': val_acc,'status': STATUS_OK, 'Trained_Model': model} 

  #returning a dictionary that contains -ve of accuracy, status of the optimization process, and the trained model

### Running the hyperoptimization process

In [44]:
trials_rf=Trials()
fmin(partial(rf_model_obj,Training_features=X_train_std,Training_class=y_train),rf_space,algo=tpe.suggest,max_evals=20,trials=trials_rf)

# extracts all valid iterations of the hyperoptimization 
valid_trial_list_rf= [trial for trial in trials_rf
                            if STATUS_OK == trial['result']['status']]
    
# extracts obj. function value in all valid iterations of the hyperoptimization 
losses_rf = [ float(trial['result']['loss']) for trial in valid_trial_list_rf]
print(losses_rf)

100%|███████████████████████████████████████████████| 20/20 [03:07<00:00,  9.39s/trial, best loss: -0.9252304802719468]
[-0.9225454016375011, -0.9229289842995648, -0.9236965905233038, -0.9240792913861444, -0.9244633149478195, -0.9244624331485963, -0.922929866098788, -0.9210106300896349, -0.9225458425371128, -0.9217786772129853, -0.9240792913861443, -0.9252295984727238, -0.9240788504865328, -0.9236961496236922, -0.9240788504865328, -0.9210110709892464, -0.9236965905233038, -0.9252304802719468, -0.924462874048208, -0.9229294251991763]


### Best hyperparameters

In [51]:
# find the one with lowest obj. function
index_having_minumum_loss_rf = np.argmin(losses_rf)
best_trial_obj_rf = valid_trial_list_rf[index_having_minumum_loss_rf]
best_model_rf = best_trial_obj_rf['result']['Trained_Model']   
print("Best hyper parameter values are")
print(best_model_rf)

Best hyper parameter values are
RandomForestClassifier(criterion='log_loss', max_features=3, min_samples_leaf=2,
                       n_estimators=680, n_jobs=4)


### Train random forest with estimated parameters

In [54]:
rf_model = RandomForestClassifier(n_estimators=680,
                          criterion='log_loss', 
                          max_features = 3, 
                          min_samples_leaf = 2, n_jobs=4
                        )
rf_model.fit(X_train_std,y_train)

RandomForestClassifier(criterion='log_loss', max_features=3, min_samples_leaf=2,
                       n_estimators=680, n_jobs=4)

In [55]:
pred_classes = rf_model.predict(X_test_std) # predicted class labels
print('testing accuracy',accuracy_score(y_test,pred_classes)) 

testing accuracy 0.9217791411042945
